## 吴恩达老师机器学习课后练习————神经网络(黄海广博士版本)

#### 此练习是基于5000个feature & label的手写体识别神经网络模型；

#### 我们通过反向传播算法实现神经网络代价函数和梯度计算的正则化和非正则化版本，并实现随机权重初始化。

In [13]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt

In [47]:
data = sio.loadmat('ex4data1.mat')

X = data['X']        # (5000, 400)
y = data['y']        # (5000, 1)

##### 将包含10种不同类别的5000个标签向量化成二进制形式，sklearn库中有现成的模块，详见配图 OneHot.png

In [30]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
encoder.fit(y)
y_onehot = encoder.transform(y)        # (5000, 10)

##### 构建神经网络，输入层(400特征+1偏置单元)，1个隐藏层(25个单元+1个偏置)，输出层(10个特征单元)

##### 定义sigmoid 核函数

In [16]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

## 前向传播：400+1 ---> 25+1 ---> 10

In [17]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 @ theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 @ theta2.T
    h = sigmoid(z3)        # (5000, 10)
    return a1, z2, a2, z3, h

##### 初始化设置

In [18]:
input_size = 400
hidden_size = 25
num_labels = 10
learning_rate = 1

##### 1~2层共25×401个theta，2~3层共10×26个theta，所以共生成 25×401+10×26 个 在(-eps, +eps)之间的数作为随机初始化theta。

##### theta1截取前25×401个，theta2截取剩下的10×26个。简化：random()×2×eps-eps = 2×eps×(random()-0.5)

In [19]:
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

##### 正则化代价函数

In [20]:
def cost(params, X, y, input_size, hidden_size, num_labels, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    theta1 = np.reshape(params[ : hidden_size*(input_size+1)], (hidden_size, (input_size+1)))
    theta2 = np.reshape(params[hidden_size*(input_size+1) : ], (num_labels, (hidden_size+1)))

    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1-y[i, :]), np.log(1-h[i, :]))
        J += np.sum(first_term - second_term)

    J /= m

    J += (float(learning_rate)/(2*m)) * (np.sum(np.power(theta1[1:], 2)) + np.sum(np.power(theta2[1:], 2)))
    
    return J

## 反向传播算法：

##### 先定义 sigmoid 函数的导数

In [21]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1-sigmoid(z)))

##### 实施反向传播来计算梯度。反向传播所需的计算就是代价函数中所需的计算过程，在此拓展函数执行反向传播返回梯度和代价

##### 误差：  $\delta^{(3)}=a^{(3)}-y$，$\delta^{(2)}=(\theta^{(2)})^T\delta^{(3)}*g'(z^{(3)})$ ，$\delta^{(1)}$没有误差(输入层)

##### 则用误差表示偏导数（未正则化）： $\frac{\partial}{\partial\Theta_{ij}^{(l)}}J(\Theta)=a_{j}^{(l)} \delta_{i}^{l+1}$

##### 最后计算偏导数矩阵（含正则化）： $ \frac{\partial}{\partial\Theta^{(l)}_{ij}}=D_{ij}^{(l)}=\frac{1}{m}\Delta_{ij}^{(l)}+\frac{\lambda}{m}\Theta_{ij}^{(l)}$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ${if}\; j \neq  0$  
##### &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  $\frac{\partial}{\partial\Theta^{(l)}_{ij}}=D_{ij}^{(l)}=\frac{1}{m}\Delta_{ij}^{(l)}$  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  ${if}\; j = 0$

In [22]:
def backprop(params, X, y, input_size, hidden_size, num_labels, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    theta1 = np.matrix(np.reshape(params[ : hidden_size*(input_size+1)], (hidden_size, (input_size+1))))
    theta2 = np.matrix(np.reshape(params[hidden_size*(input_size+1) : ], (num_labels, (hidden_size+1))))
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    J = 0
    # 初始化误差
    delta1 = np.zeros(theta1.shape)    # (25, 401)
    delta2 = np.zeros(theta2.shape)    # (10, 26)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1-y[i, :]), np.log(1-h[i, :]))
        J += np.sum(first_term - second_term)

    J /= m

    J += (float(learning_rate)/(2*m)) * (np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2)))

    # perform backpropagation
    for t in range(m):
        a1t = a1[t, :]    # (1, 401)
        z2t = z2[t, :]    # (1, 25)
        a2t = a2[t, :]    # (1, 26)
        ht = h[t, :]      # (1, 10)
        yt = y[t, :]      # (1, 10)

        d3t = ht - yt        # 最后一层的误差 delta    (1, 10)

        z2t = np.insert(z2t, 0, values=np.ones(1))    # (1, 26)
        d2t = np.multiply((theta2.T @ d3t.T).T, sigmoid_gradient(z2t))    # 第二层误差 (1, 26)

        delta1 = delta1 + d2t[:,1:].T @ a1t
        delta2 = delta2 + d3t.T @ a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the grandient regularization term
    delta1[:, 1:] = delta1[:, 1:] + (theta1[:, 1:]*learning_rate) / m
    delta2[:, 1:] = delta2[:, 1:] + (theta2[:, 1:]*learning_rate) / m
    
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

##### 终于完成了神经网络的过程，接下来用数据集训练该神经网络，并用它进行预测。

##### 由于将代价和梯度都在反向传播函数中返回了，所以 jac=True。若fun中只返回代价，则jac=梯度。详见官方文档。

In [24]:
%%time
from scipy.optimize import minimize
# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(X, y_onehot, input_size, hidden_size, num_labels, learning_rate), method='TNC', jac=True, options={"maxiter":250})
print(fmin)

     fun: 0.3343120231391213
     jac: array([-1.32377894e-04,  7.83180650e-07,  6.39625209e-07, ...,
       -3.82279414e-05,  1.81743812e-05, -6.39246575e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 23
  status: 3
 success: False
       x: array([-0.71613203,  0.0039159 ,  0.00319813, ..., -1.9969562 ,
       -2.3056285 ,  0.97026344])
Wall time: 3min 24s


##### 由于目标函数不太可能完全收敛，我们对迭代次数进行了限制。总代价已经下降到0.5以下，这是算法正常工作的一个很好的指标。 我们使用这些参数，通过网络传播，获得一些预测。

##### 使用找到的参数，通过前向传播获得预测

In [45]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size*(input_size+1)], (hidden_size, (input_size+1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size*(input_size+1):], (num_labels, (hidden_size+1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

y_pred = np.array(np.argmax(h, axis=1) + 1)        # 很巧妙，详见配图 onehot下标.png

In [46]:
from sklearn.metrics import classification_report
print(classification_report(y, y_pred))

             precision    recall  f1-score   support

          1       0.99      0.99      0.99       500
          2       0.99      1.00      0.99       500
          3       0.99      0.99      0.99       500
          4       1.00      0.99      0.99       500
          5       1.00      0.99      0.99       500
          6       1.00      0.99      1.00       500
          7       0.99      0.99      0.99       500
          8       1.00      1.00      1.00       500
          9       0.99      0.99      0.99       500
         10       0.99      1.00      1.00       500

avg / total       0.99      0.99      0.99      5000



##### 下一章开始学习支持向量机